In [3]:
%matplotlib inline
import re
import spacy
import scipy
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from stop_words import get_stop_words
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Разреженные признаки

**Примеры** источников разреженных признаков:

* Категориальные признаки 
* Текст

## Категориальные признаки

Категориальные признаки, также могут упоминаться, как факторные или номинальные признаки. 

**Примеры** категориальных признаков: 

* пол
* страна проживания
* номер группы

и т.п.

Ясно, что для компьютерной обработки вместо "понятного для человека" значения (в случае страны — "Russia", "GB", "France" и т.п.) хранят числа. Далее обсудим, как получать подобное вектор-признаки и в каком формате их хранить.

Рассмотрим таблицу ```winemag.csv```, которая содержит описания вин.

In [4]:
data = pd.read_csv('winemag.csv', index_col=0, na_filter=False)
data.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,,Provence red blend,Domaine de la Bégude


В таблице 10 столбцов с признаками. Какие из них являются категориальными? 

Во-первых, это должны быть столбцы содержащие текстовые значения. Следовательно, в качестве кандидатов остаются: ```country```, ```description```, ```designation```, ```province```, ```region_1```, ```region_2```, ```variety``` и ```winery```.

Во-вторых, столбцы с небольшим числом уникальных значений:

In [73]:
for name in ['country', 'description', 'designation', 'province',
             'region_1', 'region_2', 'variety', 'winery']:
    print('%s: %d'%(name, data[name].nunique()))

country: 49
description: 97821
designation: 30622
province: 456
region_1: 1237
region_2: 19
variety: 632
winery: 14810


Итак, страна-производитель является категориальным признаком.

Самым очевидным способом кодирования является - **one-hot-кодирование**: для кодируемого категориального признака создаются $N$ новых признаков, где $N$ - число категорий. Каждый $i$-й новый признак - бинарный характеристический признак $i$-й категории.

Например, страна-производитель является категориальным признаком. Воспользуемся [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) и [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) для преобразования названий стран в one-hot-вектор.

In [59]:
countries = data.country
countries = LabelEncoder().fit_transform(countries)
countries = OneHotEncoder().fit_transform(countries[:, np.newaxis])

type(countries)

scipy.sparse.csr.csr_matrix

## Разреженные матрицы

После кодирования мы получили разреженную матрицу признаков. Существует много типов разреженных матриц, каждый из которых предоставляет разные гарантии на операции.

* ```scipy.sparse.coo_matrix```
* ```scipy.sparse.csc_matrix```
* ```scipy.sparse.csr_matrix```
* ```scipy.sparse.bsr_matrix```
* ```scipy.sparse.lil_matrix```
* ```scipy.sparse.dia_matrix```
* ```scipy.sparse.dok_matrix```

Подробнее про [устройство разреженых матрицы](http://www.netlib.org/utk/people/JackDongarra/etemplates/node372.html)

### scipy.sparse.coo_matrix

* Используется как хранилище данных
* Поддерживает быструю конвертацию в любой формат
* Не поддерживает индексацию
* Поддерживает ограниченый набор арифметических операций

### scipy.sparse.csc_matrix

* Хранит данные поколоночно
* Быстрое получение значений отдельных колонок

### scipy.sparse.csr_matrix

* Хранит данные построчно
* Быстрое получение значений отдельных строк

### scipy.sparse.bsr_matrix

* Подходит для разреженных матриц с плотными подматрицами

### scipy.sparse.lil_matrix

* Подходит для создания разреженных матриц поэлементно
* Для последующих матричных операций лучше сконвертировать в ```csr_matrix``` или ```csc_matrix```

Библиотека ```scipy.sparse``` содержит методы, позволяющие работать с разреженными матрицами. Подробнее про операции с разрежеными матрицами на сайте [scipy](https://docs.scipy.org/doc/scipy/reference/sparse.html).

## Предсказание оценки вин

В качестве категориальных признаков возьмём: country, province и variety

Попробуем предсказать оценку выставленную винам. Оценки в таблицы варьируются от 80 до 100.

In [172]:
Y = data.points.values
countries = OneHotEncoder().fit_transform(LabelEncoder().\
                                          fit_transform(data.country)[:, np.newaxis])
provinces = OneHotEncoder().fit_transform(LabelEncoder().\
                                          fit_transform(data.province)[:, np.newaxis])
varieties = OneHotEncoder().fit_transform(LabelEncoder().\
                                          fit_transform(data.variety)[:, np.newaxis])
features = [('country', countries), ('province', provinces), ('variety', varieties)]

names = []
accuracy_scores = []
for subset_features in tqdm(itertools.chain(*[list(itertools.combinations(features, n)) 
                                              for n in range(1, 4)])):
    subset_names, subset_features = zip(*subset_features)
    names.append('; '.join(subset_names))
    
    X = scipy.sparse.hstack(subset_features)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
    
    lr = LogisticRegression().fit(X_train, Y_train)
    Y_pred = lr.predict(X_test)

    accuracy_scores.append(accuracy_score(Y_pred, Y_test))

pd.DataFrame({'Accuracy':accuracy_scores}, index=names)

7it [01:02, 10.62s/it]


,Accuracy
country,0.144478
province,0.155299
variety,0.146787
country; province,0.153111
country; variety,0.153392
province; variety,0.161825
country; province; variety,0.161885


## Извлечение признаков из текстов

Перед тем как работать с текстом, его необходимо токенизировать - разбить на отдельные токены. В качестве токенов могут выступать слова, фразы, предложений и т.п. Токенизировать текст можно  помощью регулярных выражений или готовых токенизаторов. 

После токенизации нужно привести текст к нормальной форме. Речь идет о [стемминге и/или лемматизации](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) - это схожие процессы, используемые для обработки словоформ.

Для работы лемматизации английского текста можно воспользоваться библиотекой [SpaCy]( https://spacy.io/):

In [82]:
nlp = spacy.load('en')
description_lemma = [' '.join([token.lemma_ for token in nlp(text)]) 
                     for text in tqdm(data.description)]

100%|██████████| 150930/150930 [1:41:14<00:00, 24.85it/s]  


In [83]:
description_lemma[0]

'this tremendous 100 % varietal wine hail from oakville and be age over three year in oak . juicy red - cherry fruit and a compelling hint of caramel greet the palate , frame by elegant , fine tannin and a subtle minty tone in the background . balanced and rewarding from start to finish , -PRON- have year ahead of -PRON- to develop further nuance . enjoy 2022–2030 .'

### Bag of Words

Cоздаем вектор длиной в словарь, для каждого слова считаем количество вхождений в текст и подставляем это число на соответствующую позицию в векторе.

Построим модель BOW с помощью [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [108]:
vectorizer = CountVectorizer().fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Размер словаря: 26383
Top-10 слов: and; the; be; of; with; pron; this; wine; flavor; in


Видно, что большая часть из топ-10 слов является не информативными - стоп-словами. Что бы они не участвовали в представление, в конструктор CountVectorizer в качестве параметра можно передать список стоп-слов:

In [110]:
stop_words = get_stop_words('en')
vectorizer = CountVectorizer(stop_words=stop_words).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Размер словаря: 26290
Top-10 слов: pron; wine; flavor; fruit; finish; cherry; aroma; tannin; dry; acidity


Чтобы сжать векторное представление, можно "отбросить" редкие слова:

In [113]:
vectorizer = CountVectorizer(stop_words=stop_words, min_df=3).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()
print('Размер словаря: %d'%len(vocabulary))

description_count = vectorizer.transform(description_lemma)
description_count

Размер словаря: 16143


<150930x16143 sparse matrix of type '<class 'numpy.int64'>'
	with 3750840 stored elements in Compressed Sparse Row format>

### Tf-Idf

Cлова, которые редко встречаются в корпусе (во всех рассматриваемых документах этого набора данных), но присутствуют в этом конкретном документе, могут оказаться более важными. Тогда имеет смысл повысить вес более узкотематическим словам, чтобы отделить их от общетематических. Этот подход называется [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf).

Значение Tf-Idf для каждого пары документ-слово состоит из двух компонент:
* Term frequency — логарифм встречаемости слова в документе

$$tf(t, d) = \log n_{t,d}$$

* Inverse Document frequency — логарифм обратной доли документов в которых встретилось данное слово

$$idf(t, D) = \log \frac{ \mid D \mid}{\mid \{ d_i \in D \mid t \in d_i \} \mid}$$

* Tf-Idf — кобминация tf и idf

$$ TfIdf(t, d, D) = tf(t, d) * idf(t, D)$$

In [115]:
vectorizer = TfidfVectorizer(stop_words=stop_words).fit(description_lemma)
vocabulary = vectorizer.get_feature_names()

description_tfidf = vectorizer.transform(description_lemma)
top_tokens, _ = zip(*sorted(zip(vocabulary, description_count.sum(axis=0).getA1()), 
                            key=lambda x: x[1], reverse=True)[:10])
print('Top-10 слов: %s'%'; '.join(top_tokens))

Top-10 слов: hoed; nonnenberg; consulting; coutinel; congenial; blessing; alderbrook; marriage; charlemagne; 50g


## Предсказание оценки вин

Добавляем к категориальным признакам, признаки извлечённые из описаний сомелье.

In [189]:
accuracy_scores = []
for description in [description_count, description_tfidf]:

    X = scipy.sparse.hstack([countries, provinces, varieties, description])
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
    
    lr = LogisticRegression().fit(X_train, Y_train)
    Y_pred = lr.predict(X_test)
    
    accuracy_scores.append(accuracy_score(Y_pred, Y_test))

pd.DataFrame({'Accuracy':accuracy_scores}, index=['BOW', 'TfIdf'])

,Accuracy
BOW,0.385287
TfIdf,0.325215


### BM25

Метод вычисления весов ```Okapi BM25```, который развивает идею Tf-Idf, учитывая длину документов в $tf(t, d)$.

$$tf(t, d) = \frac{(k_1 + 1) * n_{t,d}}{k_1 * (1 - b + b * \frac{L_d}{L_{ave}}) + n_{t,d}}$$

где
* $k_1$ и $b$ - свободные коэффициенты, обычно их выбирают как $k_1=2.0$ и $b=0.75$
* $L_d$ - длина документа $d$
* $L_{ave}$ - средняя длина документа в коллекции

## Vowpal Wabbit

Довольно часто бывает, что данных очень много и обучаться приходится на выборках, которые не помещаются в память. А также, довольно часто хорошее качество можно получить благодаря простым линейным моделям, при условии, что были хорошо отобраны и сгенерированы признаки. 

Важным достоинством линейных методов является то, что при обучении можно добиться того, что настройка параметров алгоритма (т.е. этап обновления весов) будет производится каждый раз при добавлении нового обьекта. Данные методы машинного обучения в литературе часто также называют Online Machine Learning. При этом не нужно хранить все обьекты одновременно в памяти теперь просто не нужно.

На сегодняшний день одной из самых известных реализаций таких методов является пакет [Vowpal Wabbit](https://github.com/JohnLangford/vowpal_wabbit):

* Можно обучать только линейные модели. Увеличивать качество методов, можно за счет добавления новых признаков и подгонки функции потерь

* Обучающая выборка обрабатывается с помощью стахостического оптимизатора, благодаря чему можно обучаться на выборках, которые не помещаются в память

* Можно обрабатывать большое количество признаков за счет их хэширования (так называемый hashing trick), бладаря чему можно обучать модели даже в случаях, когда полный набор весов просто не помещается в памяти

* Поддерживается режим активного обучения, при котором обьекты обучающей выборки можно подавать даже с нескольких машин по сети

* Обучение может быть распараллелено на несколько машин

### Установка

* Ubuntu - ```apt-get install vowpal-wabbit```
* Mac OS - ```port install vowpal_wabbit```
* Windows - скачать установочник [тут](https://github.com/eisber/vowpal_wabbit/releases)

### Формат данных

Label [weight] |Namespace Feature ... |Namespace ...

* ```Label``` - метка класса для задачи классификации или действительное число для задачи регрессии
* ```weight``` - вес объекта, по умолчанию у всех 1
* ```Namespace``` - все признаки разбиты на области видимости, может использоваться для раздельного использования или создания квадратичных признаков между областями
* ```Feature``` - ```string[:value]``` или ```int[:value]``` строки будут хешированы, числа будут использоваться как индекс в векторе признаков. ```value``` по умолчанию равно $1$

### Параметры

**Hashing trick**

Вводится функция $h$, с помощью которой получается индекс для записи значения в вектор признаков объекта.

$$h : F \rightarrow \{0, \dots, 2^b - 1\}$$

С помощью ```--b``` можно задавать размер области значений хеш-функции. Чем больше значение ```b```, тем меньше вероятность получить коллизии при хешировании признаков.

**Оптимизация**

Может использовать ```SGD``` или ```L-BFGS``` (квази-ньютоновский метод второго порядка, подробнее про работу [оптимизации](http://aria42.com/blog/2014/12/understanding-lbfgs))

По умолчанию используется ```SGD```. ```L-BFGS``` включается с помощью ```--bfgs```, работает гораздо медленнее и подходит только для выборок небольшого размера. 

Количество проходов по данным для ```SGD``` задаётся с помощью параметра ```--passes```

**Параметры оптимизации**

Обновление весов происходит на каждом объекте:
<br/>

$$w_{t+1} = w_{t} + \eta_t \nabla_{w}\ell(w_{t}, x_{t})$$
$$\eta_t = \lambda d^k \left( \frac{t_0}{t_0 + t} \right)^p$$

где $t$ - номер объекта при обучении, $k$ - номер эпохи. Остальные параметры задаются следующим образом:

* $\lambda$: ```-l```
* $d$: ```--decay_learning_rate```
* $t_0$: ```--initial_t```
* $p$: ```--power_t```

**Функция потерь** задаётся через ```--loss_function```

**Регуляризация** задаётся через два флага ```--l1``` и ```--l2```

**Квадратичные признаки**

* ```-q ab``` — создаёт квадратичные признаки, перемножая все признаки из областей видимости, названия которых начинаются на букву __a__ и на букву __b__
* ```--ignore a``` — игнорирует все признаки из области видимости, название которой начинается на букву __a__

**Режим демона**

* ```--daemon``` — запускает __vw__ в режиме сервиса на порту, который можно задать с помощью ```--port```
* Позволяет обучать модель и/или применять модель по сети

In [116]:
!vw -h | head -n10

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 


In [202]:
data_vw = pd.DataFrame({'class': Y-80, 'description':[' '.join(re.findall(r'\w+', d)) 
                                                      for d in description_lemma]})
data_train, data_test = train_test_split(data_vw)
data_train.to_csv('data.train.vw', sep='|', header=None, index=False)
data_test.to_csv('data.test.vw', sep='|', header=None, index=False)

In [203]:
!vw -d data.train.vw  -f model.vw --loss_function logistic --oaa 21 --quiet --passes 100 -c -k

* ```-d``` — откуда брать данны для обучения
* ```-f``` — куда сохранять модель
* ```--passes``` — максимальное количество проходов по выборке
* ```-c``` — создавать файл с кешем, необходимо указывать, если используется ```--passes```
* ```-k``` — перед запуском очищать кеш

In [204]:
!vw -d data.test.vw -i model.vw -r output.csv --loss_function logistic --quiet 

* ```-i``` — путь до готовой модели
* ```-t``` — не обучаться, только вернуть предсказания
* ```-p``` — путь до файла для записи предсказаний
* ```--quite``` — не выводить никакую информацию в консоль